In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,NOFX,Jamaica's Alright If You Like Homophobes,"Jamaica's alright, if you wanna get in the mid..."
1,Venom,Playtime,Told you the story 'bout a teather's pet \r\n...
2,Rod Stewart,Having A Party,"Yeah, clap your hands \r\n \r\nWe're having ..."
3,Clash,Tainted Love,Sometimes I feel I've got to \r\nRun away I'v...
4,Counting Crows,A Good Year For The Roses,I can hardly stand the sight of lipstick \r\n...
5,Kenny Chesney,Scare Me,I'm not afraid of lonely \r\nI've gotten good...
6,King Diamond,Help!!!,Help... help... can anybody help me now? \r\n...
7,Kid Rock,"Jackson, Mississippi",And when the sunshine showed \r\nHer face I f...
8,Alice In Chains,Choke,Before you ask for something better \r\nYou s...
9,David Pomeranz,Tryin' To Get The Feeling Again,"Doctor, my woman \r\nIs comin' back home late..."


In [10]:
df['text'][0]

"Jamaica's alright, if you wanna get in the middle of a knife fight  \r\nJamaica's alright, if you wanna get aids or dysentery  \r\nJamaica's alright, if you wanna be mugged or murdered  \r\nJamaica's alright, if you're a misogynist, or a homophobe  \r\n  \r\nJamaica's alright, if you like only one kind of music  \r\nJamaica's alright, if you like small bananas and spiders  \r\nJamaica's alright, if you wanna buy drugs from your hotel room  \r\nJamaica's alright, if you want your hand lopped off by a machete\r\n\r\n"

In [12]:
df = df.sample(5000)

In [13]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [14]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [18]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
nltk.download('punkt_tab')

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt_tab to /home/sarthak-
[nltk_data]     bhardwaj/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.        , 0.05276361, 0.03310318, ..., 0.00559959, 0.14111787,
       0.07456491], shape=(5000,))

In [24]:
def recommendation(song_df):
    # Check if any rows match the condition
    if not df[df['song'] == song_df].empty:
        idx = df[df['song'] == song_df].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
        
        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)
            
        return songs
    
    else:
        # Handle the case when no matching rows are found
        print(f"No data found for song: {song_df}")

In [25]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))